In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import itertools
import sqlalchemy

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
cursor = session.connection.cursor()

In [ ]:
SELECT INDEX, FIRST_NAME, LAST_NAME, PHONE, EMAIL, ADDRESS_1, ADDRESS_2, CITY, STATE, ZIPCODE, ZIP_PLUS_FOUR_CODE, INFO_TO_REMOVE FROM SAMPLE_TABLE
--SELECT * FROM SAMPLE_TABLE
WHERE DUMPED = FALSE

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import sqlite3
import os

# Then, we can use the python name to turn cell2 into a Pandas dataframe
df = getDNMList.to_pandas()
newdf = pd.DataFrame(columns=df.columns)
abbreviations = {
        "drive": ["dr", "drv", "dve", "dr."],
        "street": ["st", "str", "st.", "strt"],
        "avenue": ["ave", "av", "av.", "avenu"],
        "boulevard": ["blvd", "boul", "bvd", "boulv"],
        "road": ["rd", "rd.", "rode"],
        "lane": ["ln", "ln.", "lne"],
        "terrace": ["ter", "ter.", "terr"],
        "place": ["pl", "pl.", "plc"],
        "court": ["ct", "ct.", "cour"],
        "south": ['s'],
        'east': ['e'],
        'west': ['w'],
        'north': ['n'],
        'southeast': ['se'],
        'southwest': ['sw'],
        'northeast': ['ne'],
        'northwest': ['nw']
    }
cursor = session.connection.cursor()

#st.write(df.size)
    # Create a reverse lookup dictionary to map values back to their keys
reverse_abbreviations = {abbr: key for key, abbrs in abbreviations.items() for abbr in abbrs}
for x in range(0,int(df.size/12)):
    #st.write(x)
    # Split the address into parts
    add1 = df['ADDRESS_1'][x].lower()
    #st.write(add1)
    parts = add1.split()
    #st.write(parts)
    # Create a list to hold possible variations of each part
    possible_variations = []

    # Iterate over each part of the address
    for part in parts:
        # Add the original part to the variations
        variations = [part]
        # If the part has known abbreviations or misspellings, add those to the variations
        if part in abbreviations:
            variations.extend(abbreviations[part])
        elif part in reverse_abbreviations:
            # Add the original key and all of its abbreviations/misspellings
            key = reverse_abbreviations[part]
            variations.append(key)
            variations.extend(abbreviations[key])
        possible_variations.append(variations)
    
    # Use itertools.product to create all possible combinations of the variations
    all_combinations = list(itertools.product(*possible_variations))
    
    # Join each combination back into a single string
    all_variations = [' '.join(combination) for combination in all_combinations]
    for var in all_variations:
        cur_row = df.loc[x].copy()
        cur_row.transpose()
        #st.write(cur_row['ADDRESS_1'])
        cur_row['ADDRESS_1'] = var
        #st.write(cur_row['ADDRESS_1'])
        if var not in df['ADDRESS_1']:
            newdf.loc[-1] = cur_row  # adding a row
            newdf.index = newdf.index + 1  # shifting index
            newdf = newdf.sort_index()
    
        #st.write(df)
    newdf=newdf.drop_duplicates(subset=['ADDRESS_1'], keep='first')

    #st.write(newdf)
    #st.write(all_variations)
#newdf['ID'] = 0
#for i in range(0,len(newdf)+1):
#    newdf.at[i, 'ID'] = i

st.write(newdf)

In [ ]:
updated = []

for i in range(1, max(newdf['INDEX'])+1):
    for row in newdf.itertuples():
        #st.write(row)
        #st.write(i)
        if row[1] == i:
            #st.write(row)
            bQuery = f"""SELECT CUSTOMER_NUMBER, CUSTOMER_FIRST_NAME, CUSTOMER_LAST_NAME FROM SAMPLE_INFO_TABLE """
            nepaWhere = f"""WHERE CUSTOMER_FIRST_NAME = upper('{row[2]}') and CUSTOMER_LAST_NAME = upper('{(row[3])}') and CUSTOMER_PHONE_NUMBER = '{row[4]}' and CUSTOMER_EMAIL_ADDRESS = '{row[5]}'
                            AND HOUSEHOLD_ADDRESS_1 = upper('{(row[6])}') AND HOUSEHOLD_ADDRESS_2 = upper('{(row[7])}') and HOUSEHOLD_CITY = upper('{(row[8])}') 
                            and HOUSEHOLD_STATE_CODE = upper('{(row[9])}') and zip_code = '{row[10]}'"""
            neaWhere = f"""WHERE CUSTOMER_FIRST_NAME = upper('{row[2]}') and CUSTOMER_LAST_NAME = upper('{(row[3])}') and CUSTOMER_EMAIL_ADDRESS = '{row[5]}'
                            AND HOUSEHOLD_ADDRESS_1 = upper('{(row[6])}') AND HOUSEHOLD_ADDRESS_2 = upper('{(row[7])}') and HOUSEHOLD_CITY = upper('{(row[8])}') 
                            and HOUSEHOLD_STATE_CODE = upper('{(row[9])}') and zip_code = '{row[10]}'"""
            npaWhere = f"""WHERE CUSTOMER_FIRST_NAME = upper('{row[2]}') and CUSTOMER_LAST_NAME = upper('{(row[3])}') and CUSTOMER_PHONE_NUMBER = '{row[4]}'
                            AND HOUSEHOLD_ADDRESS_1 = upper('{(row[6])}') AND HOUSEHOLD_ADDRESS_2 = upper('{(row[7])}') and HOUSEHOLD_CITY = upper('{(row[8])}') 
                            and HOUSEHOLD_STATE_CODE = upper('{(row[9])}') and zip_code = '{row[10]}'"""
            naWhere =f"""WHERE CUSTOMER_FIRST_NAME = upper('{row[2]}') and CUSTOMER_LAST_NAME = upper('{(row[3])}') 
                            AND HOUSEHOLD_ADDRESS_1 = upper('{(row[6])}') AND HOUSEHOLD_ADDRESS_2 = upper('{(row[7])}') and HOUSEHOLD_CITY = upper('{(row[8])}') 
                            and HOUSEHOLD_STATE_CODE = upper('{(row[9])}') and zip_code = '{row[10]}'"""
            aWhere = f"""WHERE HOUSEHOLD_ADDRESS_1 = upper('{(row[6])}') AND HOUSEHOLD_ADDRESS_2 = upper('{(row[7])}') and HOUSEHOLD_CITY = upper('{(row[8])}') 
                            and HOUSEHOLD_STATE_CODE = upper('{(row[9])}') and zip_code = '{row[10]}'"""
            
            final = pd.DataFrame()
            found = False

            nepaQ = pd.DataFrame(cursor.execute(bQuery + nepaWhere))
            if not nepaQ.empty:
                st.write(nepaQ)
                st.write("found using nepa")
                final = nepaQ
                found = True
                
                
            neaQ = pd.DataFrame(cursor.execute(bQuery + neaWhere))
            if not neaQ.empty and not found:
                st.write(neaQ)
                st.write("found using nea")
                final = neaQ
                found = True
                
                
            npaQ = pd.DataFrame(cursor.execute(bQuery + npaWhere))
            if not npaQ.empty and not found:
                st.write(npaQ)
                st.write("found using npa")
                final = npaQ
                found = True
                
                
            naQ = pd.DataFrame(cursor.execute(bQuery + naWhere))
            if not naQ.empty and not found:
                st.write(naQ)
                st.write("found using na")
                final = naQ
                found = True
                
                
            aQ = pd.DataFrame(cursor.execute(bQuery + aWhere))
            if not aQ.empty and not found:
                st.write(aQ)
                st.write("found using a")
                final = aQ
                found = True
                
            #elif not found:
                #st.write("Customer not found in DB")

            if found and not final.empty:
                updated.append(row[1])
                cNum = final[0][0]
                st.write(cNum)
                cursor.execute(f"""UPDATE SAMPLE_INFO_TABLE SET 
                         Address_Suppression_Ind = 'P'
                        , Do_Not_Mail_Indicator = 'Y'
                        , Customer_Last_Modified_Date = CURRENT_DATE
                        , Customer_Last_Modified_Source = 'CPRA'
                        , Last_Update_Timestamp = CURRENT_TIMESTAMP
                        WHERE Customer_Number IN ({cNum});
                        """)
                st.write(str(cNum) + " updated")



In [ ]:
for i in updated:
    cursor.execute(f"""UPDATE SAMPLE_TABLE SET 
                        DUMPED = TRUE,
                        DUMPED_DT = GETDATE()
                        WHERE index = {i};
                        """)